# Conv_BoDiEs 실제 이미지 시험

## 1. 라이브러리 import 및 GPU 연결결

In [ ]:
import csv
import torch
import os
from torchvision import transforms
from PIL import Image
from pathlib import Path
import torch.nn as nn

# Get CPU or GPU device for training
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")
if torch.cuda.is_available():
    print(f'Device name: {torch.cuda.get_device_name(0)}')

## 2. 모델 정의

In [ ]:
# 모델 정의
class Conv_BoDiEs(nn.Module):
    def __init__(self):
        super(Conv_BoDiEs, self).__init__()

        self.conv1 = nn.Sequential(
            nn.Conv2d(1, 256, kernel_size=5, padding=2),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )

        self.conv2 = nn.Sequential(
            nn.Conv2d(256, 128, kernel_size=5, padding=2),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )

        self.conv3 = nn.Sequential(
            nn.Conv2d(128, 64, kernel_size=5, padding=2),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )

        self.conv4 = nn.Sequential(
            nn.Conv2d(64, 32, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )

        self.flatten = nn.Flatten()

        # Input size is (b, 32, 12, 12) after conv4 and maxpool
        flattened_size = 32 * 12 * 12

        self.fc1 = nn.Sequential(
            nn.Linear(flattened_size, 128),
            nn.ReLU()
        )

        self.fc2 = nn.Linear(128, 16)

    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.conv3(x)
        x = self.conv4(x)
        x = self.flatten(x)
        x = self.fc1(x)
        x = self.fc2(x)
        return x

model = Conv_BoDiEs().to(device)


## 3. 라벨정의 및 이미지 전처리리

In [ ]:
# 신체 부위 라벨 지정 / 순서 매우 중요요
body_parts = [
    "chest circ", "waist circ", "pelvis circ", "neck circ", "bicep circ", 
    "thigh circ", "knee circ", "arm length", "leg length", "calf length", 
    "head circ", "wrist circ", "arm span", "shoulders width", "torso length", 
    "inner leg"
]

# 시험 이미지 경로 지정정
image_path = 'real_test_image_original/test_200/3rd_try/black_background535.jpg'

model_path = 'bodymodel/male/Conv_BoDiEs_male_grayscale.pth'
result_file_path = 'result/real_gray_scale/T_200_try3_B_535.csv'

# 이미지 전처리 정의
transform = transforms.Compose([
    transforms.Resize((200, 200)),
    transforms.ToTensor()
])

# 단일 이미지 전처리
def preprocess_image(image_path):
    image = Image.open(image_path).convert('L')  # Grayscale 변환
    image = transform(image)
    image = image.unsqueeze(0)  # 배치 차원 추가
    return image

# 이미지 전처리
input_image = preprocess_image(image_path).to(device)

## 4. 이미지에 대한 예측 값 출력 및 저장장

In [ ]:
# 모델 평가 모드 설정 및 예측 수행
model.load_state_dict(torch.load(model_path))
model.eval()
with torch.no_grad():
    output = model(input_image)

# 예측 값 가져오기
predicted_values = output.cpu().numpy()[0]  # 16개의 예측 값

# 결과 출력
print(f'Prediction for the image: {predicted_values}')

# 결과 저장 경로 생성 (필요시)
Path(result_file_path).parent.mkdir(parents=True, exist_ok=True)

# CSV 파일로 결과 저장
with open(result_file_path, mode='w', newline='') as file:
    writer = csv.writer(file)

    # 첫 번째 행: 헤더
    writer.writerow(['Body Part', 'Prediction'])

    # 두 번째 행부터: 각 부위와 값
    for body_part, value in zip(body_parts, predicted_values):
        writer.writerow([body_part, value])

print(f'Result saved to {result_file_path}')